In [3]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
import math as m
import random
import Bandit as bnd

In [34]:
def KNN_UCB(bandit,n,cv, theta, phi): ##cv:context vector
    n_arms = bandit.k #number of arms
    actions = []
    rewards = []
    for a in range(n_arms): ##play each arm once regardless of context
        reward = bandit.pull(a)
        rewards.append(reward)
        actions.append(a)
    for t in range(n_arms,n,1):
        distances= cv[:t] - cv[t] * np.ones(t) 
        k_a = np.zeros(n_arms)
        index = np.zeros(n_arms) ## to store I_t,k
        for a in range(n_arms): ##evaluate k_a for all a
            U = np.zeros(t-1)
            for k in range(1,t, 1): ##calculate uncertainty on [t-1]
                k_least = np.argpartition(distances, k-1)[:k] ## indices of k nearest neighbours
                N = sum(np.array(actions)[k_least] == a) ##N_t,k
                U[k-1] = m.sqrt(theta* m.log(t)/N) + phi(t) * distances[k_least[-1]] if (N!=0) else phi(t) * distances[k_least[-1]]
            k_a[a] = np.argmin(U) + 1
            k_least = np.argpartition(distances, int(k_a[a]-1))[:int(k_a[a])]
            N = sum(np.array(actions)[k_least] == a)
            S = sum(np.array(rewards)[k_least] * (np.array(actions)[k_least] == a))
            f_hat = S/N if (N!=0) else 0
            index[a] = f_hat + m.sqrt(theta* m.log(t)/N) + phi(t) * distances[k_least[-1]] if(N!=0) else f_hat + phi(t) * distances[k_least[-1]]
        arm = np.argmax(index)
        reward = bandit.pull(arm)
        actions.append(arm)
        rewards.append(reward)
    return actions, rewards, bandit.regret

In [38]:
bandit = bnd.GaussianBandit([-0.1,0])
def phi(t):
    return 1
actions, rewards, regret = KNN_UCB(bandit, 40, np.ones(40), 0.5, phi)
actions, rewards

([0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [-0.18563484170363337,
  -1.4284131312662116,
  1.717989623133634,
  -1.2665963385809242,
  0.6676343230197493,
  -1.0966331073685769,
  -0.9184114874522402,
  1.563988046712201,
  -0.5659150213463929,
  -1.540270341308198,
  0.6102244477712109,
  0.22167134912566264,
  -0.5865351418726732,
  0.6789236566793903,
  -0.5065408415025394,
  0.27766734323181286,
  0.0777794951335877,
  1.256908800796717,
  0.14495111799781224,
  0.8328283414477224,
  -0.04159408661646156,
  0.1674374909937518,
  -1.069581024913078,
  -0.7238387640449342,
  1.1912380562677443,
  -0.5010958074973306,
  -0.8472501373287084,
  0.7707895133905591,
  -1.9160385121553942,
  -1.3360086343985929,
  1.3813899725979606,
  0.10722655600063291,
  -0.3585760225313519,
  -0.9617756519254068,
  -1.1603522985393209,
  1.62183

In [48]:
A = [-10,23,4,5,6,8,9,-3,-29,-1000]
print(np.argpartition(A, 3)[:3])
print(np.argpartition(A, 3))

[8 9 0]
[8 9 0 7 2 3 4 1 5 6]
